# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [4]:
import os
import gzip
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import urllib.request

# Download manually
def download_data():
    base_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"
    files = {
        "train_images": "train-images-idx3-ubyte.gz",
        "train_labels": "train-labels-idx1-ubyte.gz",
        "test_images": "t10k-images-idx3-ubyte.gz",
        "test_labels": "t10k-labels-idx1-ubyte.gz"
    }
    os.makedirs("data", exist_ok=True)
    for name, file in files.items():
        path = os.path.join("data", file)
        if not os.path.exists(path):
            urllib.request.urlretrieve(base_url + file, path)

def load_images(path):
    with gzip.open(path, 'rb') as f:
        f.read(16)
        data = np.frombuffer(f.read(), dtype=np.uint8).reshape(-1, 28*28)
    return data.astype(np.float32) / 255.0

def load_labels(path):
    with gzip.open(path, 'rb') as f:
        f.read(8)
        return np.frombuffer(f.read(), dtype=np.uint8)

# Custom Dataset
class FashionMNISTDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.tensor(images)
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Simple MLP model
class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

def main():
    download_data()
    train_images = load_images("data/train-images-idx3-ubyte.gz")
    train_labels = load_labels("data/train-labels-idx1-ubyte.gz")
    test_images = load_images("data/t10k-images-idx3-ubyte.gz")
    test_labels = load_labels("data/t10k-labels-idx1-ubyte.gz")

    train_dataset = FashionMNISTDataset(train_images, train_labels)
    test_dataset = FashionMNISTDataset(test_images, test_labels)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = FashionNet().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training
    for epoch in range(10):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)

            outputs = model(X)
            loss = loss_fn(outputs, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            outputs = model(X)
            _, preds = torch.max(outputs, 1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

    torch.save(model.state_dict(), "fashion_model.pth")

if __name__ == "__main__":
    main()


Epoch 1, Loss: 0.5478
Epoch 2, Loss: 0.3918
Epoch 3, Loss: 0.3564
Epoch 4, Loss: 0.3332
Epoch 5, Loss: 0.3190
Epoch 6, Loss: 0.3050
Epoch 7, Loss: 0.2921
Epoch 8, Loss: 0.2841
Epoch 9, Loss: 0.2739
Epoch 10, Loss: 0.2683
Test Accuracy: 88.55%


In [5]:
import torch
import torch.nn as nn

class FashionNet(nn.Module):
    def __init__(self):
        super(FashionNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

def load_model(path="fashion_model.pth"):
    model = FashionNet()
    model.load_state_dict(torch.load(path, map_location=torch.device("cpu")))
    model.eval()
    return model

# Example usage
if __name__ == "__main__":
    model = load_model()
    dummy = torch.rand(1, 28*28)
    output = model(dummy)
    print("Output:", output)

Output: tensor([[ -1.9625,  -4.4576,  -2.2076,  -7.0191,  -4.1300, -12.2829,   0.3512,
         -13.7079,   6.7708, -12.4100]], grad_fn=<AddmmBackward0>)
